# Make conjoint features
### Christian Roncal CMSC499A Dr. Leiserson 

### Todo: Make this a python script

## Load data

In [1]:
import pandas as pd

In [2]:
flu = pd.read_csv('flu_train.csv')
hepc = pd.read_csv('hepc_train.csv')
ebola = pd.read_csv('ebola_train.csv')

In [4]:
' '.join(flu['virusUprot'].unique())

'P03433 P03466 P03428 P03485 P08276 P03496 A4ZNP8 P31345 P03495 P03437 Q82506 Q9WPI5 P06821 P03431 P03468 P03508 P03452 Q67296 Q1K9E3 P06827 P69258 Q1K9E2 Q9DLK6 P69277 P15659 P03427 Q1K9I7 B2BUI7 B4UPA8 B4URA9 B2BUF1 I6TAG8 B4UPB3 B4UPB6 T2AAS6 B4UPB1 P04664 Q99AU3 O56264 Q6PNE5 Q0HD54 C3W5S7 D1LN35 Q8QPK2 Q5L4H4 Q1NZ49 C3W6D7 B2BU63 C9S3S8 Q6DPG0 Q91U50 Q8JR21 H9XIJ5 P03430 P05777 P03470 P05780 Q20MH0 P03454 P15682 Q1K9H5 Q89733 Q1K9H6 Q1I2B2 Q20MH3 Q6DPA0 Q6DP94 Q0A2H1 Q20MD0 Q20MD1 Q2PJP1 Q2PJP0 Q20MC8 Q6DP93 Q0A2H0 Q1K9H2 Q20MC6 Q20MH4 Q20MD2 A5Z256 Q6QT26 Q20MD4 Q20MD5 Q1K9G9 Q464B3 Q38SR2 Q20MC9 Q2RBU1 Q38SQ9 Q20NX5 Q30NB4 Q30NP4 Q6DNN3 B2BTY8 Q67020 Q5QF18 B4URF7 A4ZNR3 A8C8W5 B4URE7 Q20MH8 B4URF0 Q92900 Q99729 Q9Y224 P16403 Q96G97 P38919 Q5JVF3 Q14151 O43660 P18621 Q9UKM9 Q12906 Q92499 Q8WUD4 O60832 O75152 P39019 O75643 P51398 P41208 O75934 P62750 Q9NPA8 P46109 P63220 Q15717 P43243 Q9P013 Q7L2E3 P55265 Q9Y3I0 Q9NZI8 P35573 P42766 P14678 P51991 Q9UMS4 B4URF6 O43143 O60508 P2637

In [1]:
import numpy as np
import pickle
from Bio import SeqIO
from Bio.Alphabet import ProteinAlphabet
import json

In [2]:
D = pickle.load(open('D.pkl', 'rb'))

In [3]:
classes = dict.fromkeys(['A', 'G','V'], 1)
classes.update(dict.fromkeys(['I', 'L', 'F', 'P'], 2))
classes.update(dict.fromkeys(['Y', 'M', 'T', 'S'], 3))
classes.update(dict.fromkeys(['H', 'N', 'Q', 'W'], 4))
classes.update(dict.fromkeys(['R', 'K'], 5))
classes.update(dict.fromkeys(['D', 'E'], 6))
classes.update(dict.fromkeys(['C', 'U'], 7))

In [4]:
def protToClass(p):
    if p == 'X':
        return '-1'
    else:
        return str(classes[p])

def seqToClass(seq):
    return ''.join(list(map(protToClass, seq)))

def normalize(Fi):
    return (Fi - min(Fi)) / max(Fi)

def getFi(D, seq):
    grptoi = {p:i for i,p in enumerate(D)} # group to index mappings
    Fi = np.zeros(len(grptoi.values()))
    
    classSeq = seqToClass(seq)
#     print(classSeq)
    
    for p in D:
        Fi[grptoi[p]] += classSeq.count(''.join(p))

    return normalize(Fi)

In [5]:
def proteinize(records):
    for r in records:
        r.seq.Alphabet = ProteinAlphabet

In [6]:
def makeF(records):
    n = len(records)
    
    F = np.zeros((n, 2793 + 1))
    uniprottoi = {}
    itouniprot = {}
    
    
    for i in range(n):
        F[i,:] = np.append(getFi(D, records[i]), [1]) # add bias
        virus_name = records[i].name.split('|')[1]
        uniprottoi[virus_name] = i
        itouniprot[i] = virus_name
        
    return F, uniprottoi, itouniprot

In [7]:
virus_records = list(SeqIO.parse('virus_prots.fasta', 'fasta'))
human_records = list(SeqIO.parse('human_prots.fasta', 'fasta'))

In [8]:
proteinize(virus_records)

In [9]:
print(virus_records[0].name.split('|')[1]) # to get names of viruses

Q9WMX2


In [10]:
vfeats, vtoi, itov = makeF(virus_records)

In [11]:
'B4UPB0' in vtoi.keys()

False

In [11]:
np.savetxt('vfeats.txt', vfeats)

In [12]:
with open('vtoi.json', 'w') as f:
    json.dump(vtoi, f)
    
with open('itov.json', 'w') as f:
    json.dump(itov, f)

In [13]:
v = np.loadtxt('vfeats.txt')

In [14]:
hfeats, htoi, itoh = makeF(human_records)

(3302, 2794)


In [15]:
with open('htoi.json', 'w') as f:
    json.dump(htoi, f)
    
with open('itoh.json', 'w') as f:
    json.dump(itoh, f)

In [16]:
np.savetxt('hfeats.txt', hfeats)

In [17]:
h = np.loadtxt('hfeats.txt')

In [18]:
h.shape

(3302, 2794)